In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


url = 'https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/demand_data_exercise.csv'
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')


data_original = data.copy()


print("First few rows of the dataset:")
print(data.head())


print("\nDescriptive Statistics for 'Demand_with_Trend':")
print(data['Demand_with_Trend'].describe())


plt.figure(figsize=(12, 6))
plt.plot(data.index, data['Demand_with_Trend'], label='Demand_with_Trend')
plt.title('Demand with Trend Over Time')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


scaler = MinMaxScaler()
data['Demand_with_Trend'] = scaler.fit_transform(data[['Demand_with_Trend']])


train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]


X_train, y_train = train[['Demand_with_Trend']].values, train['Demand_with_Trend'].values
X_test, y_test = test[['Demand_with_Trend']].values, test['Demand_with_Trend'].values
X_train = X_train.reshape((X_train.shape[0], 1, 1))
X_test = X_test.reshape((X_test.shape[0], 1, 1))


train['EMA'] = train['Demand_with_Trend'].ewm(span=10, adjust=False).mean()

test['EMA'] = train['EMA'].iloc[-1]


arima_model = ARIMA(train['Demand_with_Trend'], order=(5, 1, 0)).fit()
train['ARIMA_Forecast'] = arima_model.fittedvalues
test['ARIMA_Forecast'] = arima_model.forecast(steps=len(test))


rnn_model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(1, 1)),
    Dropout(0.2),
    Dense(1)
])
rnn_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


rnn_predictions_normalized = rnn_model.predict(X_test)
rnn_predictions_rescaled = scaler.inverse_transform(rnn_predictions_normalized)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))


lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(1, 1)),
    Dropout(0.2),
    Dense(1)
])
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


lstm_predictions_normalized = lstm_model.predict(X_test)
lstm_predictions_rescaled = scaler.inverse_transform(lstm_predictions_normalized)


plt.figure(figsize=(12, 6))
plt.plot(data.index[train_size:], y_test_rescaled, label='Actual Demand', color='orange')
plt.plot(train.index, train['EMA'], label='Training EMA', color='blue')
plt.plot(test.index, test['EMA'], label='Testing EMA (Constant)', color='blue', linestyle='--')
plt.plot(test.index, test['ARIMA_Forecast'], label='ARIMA Forecast', color='green')
plt.plot(data.index[train_size:], rnn_predictions_rescaled, label='RNN Forecasted Demand', color='purple', linestyle='--')
plt.plot(data.index[train_size:], lstm_predictions_rescaled, label='LSTM Forecasted Demand', color='red', linestyle='--')
plt.title('EMA, ARIMA, RNN, and LSTM Model - Actual vs. Forecasted Demand with Trend')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


mae_ema = mean_absolute_error(y_test_rescaled, test['EMA'].values.reshape(-1, 1))
rmse_ema = np.sqrt(mean_squared_error(y_test_rescaled, test['EMA'].values.reshape(-1, 1)))
mape_ema = np.mean(np.abs((y_test_rescaled - test['EMA'].values.reshape(-1, 1)) / y_test_rescaled)) * 100


mae_arima = mean_absolute_error(y_test_rescaled, test['ARIMA_Forecast'].values.reshape(-1, 1))
rmse_arima = np.sqrt(mean_squared_error(y_test_rescaled, test['ARIMA_Forecast'].values.reshape(-1, 1)))
mape_arima = np.mean(np.abs((y_test_rescaled - test['ARIMA_Forecast'].values.reshape(-1, 1)) / y_test_rescaled)) * 100


mae_rnn = mean_absolute_error(y_test_rescaled, rnn_predictions_rescaled)
rmse_rnn = np.sqrt(mean_squared_error(y_test_rescaled, rnn_predictions_rescaled))
mape_rnn = np.mean(np.abs((y_test_rescaled - rnn_predictions_rescaled) / y_test_rescaled)) * 100


mae_lstm = mean_absolute_error(y_test_rescaled, lstm_predictions_rescaled)
rmse_lstm = np.sqrt(mean_squared_error(y_test_rescaled, lstm_predictions_rescaled))
mape_lstm = np.mean(np.abs((y_test_rescaled - lstm_predictions_rescaled) / y_test_rescaled)) * 100


print("\nEvaluation Metrics for EMA (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_ema:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_ema:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_ema:.2f}")

print("\nEvaluation Metrics for ARIMA (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_arima:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_arima:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_arima:.2f}")

print("\nEvaluation Metrics for RNN (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_rnn:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_rnn:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_rnn:.2f}")

print("\nEvaluation Metrics for LSTM (Testing Set):")
print(f"Mean Absolute Error (MAE): {mae_lstm:.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {mape_lstm:.2f}%")
print(f"Root Mean Squared Error (RMSE): {rmse_lstm:.2f}")
